# Week 3

During week 3 of the project, we worked on replacing the dino and obstacle blocks with bitmap images of a cartoon dinosaur and 3 different types of cactii to make it aesthetically pleasing. In order to do so, we first used the predefined `tft_drawBitmap()` function and successfully implemented the dino and cactii graphics in the game. However, we realized that the function is not optimized and takes a lot of CPU cycles to run. Thus, we digged into the header files and created our own optimized function for drawing bitmaps.

Next, we implemented the live score and high score functionality in the game.

---

## The Design

#### 1. The graphic images

In order to implement the graphics for the dino and cactii, we downloaded stock images of 3 different cactii, and 3 configurations of the dino - jump position, running position 1 and running position 2. 

In order from left to right: small cactus, large cactus and multiple cactii.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/smallCactus.png" style="margin:10px 50px; width: 20px; height: 40px; " >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/largeCactus.png" style="margin:10px 50px; width: 30px; height: 60px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/multiCactus.png" style="margin:10px 50px; width: 60px; height: 40px;" >
</div>

In order from left to right: dino jump, running position 1 and running position 2.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/dinoJump.png" style="margin:10px 50px; width: 44px; height: 50px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/dinoRun1.png" style="margin:10px 50px; width: 44px; height: 50px;" >
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/dinoRun2.png" style="margin:10px 50px; width: 44px; height: 50px;" >
</div>

<br>

#### 2. Converting images to bitmaps

In order to display these images in the game, we needed to convert them in a form which a microcontroller could understand. This form is known as a `bitmap`. A bitmap is basically an array of pixels where each bit of the array signifies whether a pixel is dark or light. If the pixel is light, the bit is set to 1 and if the pixel is dark, the bit is set to 0. In order to conserve space, the pixels are represented in a hexdecimal format so that each element of the array represents a series of 8 pixels.

In order to convert these PNG images in the bitmap format, we used [LCD Image Converter](https://lcd-image-converter.riuson.com/en/about/). We followed the following steps for this conversion:
- Open the desired PNG image in the software.
- Click on `Options > Conversion`.
- Set the preset to `Monochrome` and click `Show Preview`.
- This will open up a new window which displays the 1D array of the desired 2D bitmap image.

#### 3. Creating the bitmap header

The above conversion was repeated for all six images and we stored the resultant arrays in a file called `BitMap.h` which can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/BitMap.h). Every single array is of the type `const unsigned char`. We used `const` because the bitmap will remain constant throught the execution of the program and will be stored in the flash memory.

#### 4. Implementing the predefined bitmap function

The provided `tft_gfx.h` library has a predefined function called `tft_drawBitmap()` which takes in the following parameters to draw the image:
- The x-coordinate of the image
- The y-coordinate of the image
- The pointer to the first element of the bitmap array
- The width of the image
- The height of the image
- The color of the image

We implemented the code as below.

We first included the bitmap library.

```c
#include "BitMap.h"
```

<br>

We then defined `RUNNER_FRAMES` constant value and a variable `runner` which help us in creating an illusion of the dino running.

```c
#define RUNNER_FRAMES 10
char runner = 0;
```

<br>

The next step was to replace all the `tft_fillRect()` functions in the animation protothread with the `tft_drawBitmap()` function to implement the dino cactii graphics.

_Note: To implement the cactii graphics, we had to use a switch case which draws the correct bitmap for the cactii based on the variable `obsType`._

```c
switch(obsType){
    case 0: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeZer, obstacle.w, obstacle.h, ILI9340_BLACK);
            break;
    case 1: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeOne, obstacle.w, obstacle.h, ILI9340_BLACK);
            break;
    case 2: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeTwo, obstacle.w, obstacle.h, ILI9340_BLACK);
            break;
}

//After updating the obstacle parameters

switch(obsType){
    case 0: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeZer, obstacle.w, obstacle.h, HARD_COLOR);
            break;
    case 1: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeOne, obstacle.w, obstacle.h, HARD_COLOR);
            break;
    case 2: tft_drawBitmap(obstacle.x, (HEIGHT - GROUND_HEIGHT - ((obstacle.h / 2))), obsTypeTwo, obstacle.w, obstacle.h, HARD_COLOR);
            break;
}
```

<br>

After updating the player parameters, we had to draw the appropriate dino image. We used the following three conditions to draw the dino:
- If the dino is in the middle of a jump, then we draw the dino jump bitmap.
- If the `runner` variable is between 0 and `RUNNER_FRAMES` / 2, then draw the dino running position 1 bitmap.
- If the `runner` variable is between `RUNNER_FRAMES` / 2 and `RUNNER_FRAMES`, then draw the dino running position 2 bitmap.

Once the `runner` variable hits `RUNNER_FRAMES`, it will reset to 0.

```c
if(myPlayer.y > 0){
    tft_drawBitmap(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), dinoJumpUp, myPlayer.w, myPlayer.h, SOFT_COLOR);
}
else{
    if(runner > RUNNER_FRAMES / 2){
        tft_drawBitmap(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), dinoRunOne, myPlayer.w, myPlayer.h, SOFT_COLOR);
    }
    else{
        tft_drawBitmap(myPlayer.x, (HEIGHT - GROUND_HEIGHT - (myPlayer.y + (myPlayer.h / 2))), dinoRunTwo, myPlayer.w, myPlayer.h, SOFT_COLOR);
    }
}
runner = (runner + 1) % RUNNER_FRAMES;
```

<br>

After implementing the bitmaps, we tested the code out using the GUI. The result is shown below.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/BitmapUnoptimized.png" style="width: 600px; height: 405px;" >
</div>
<figure>
    <center><figcaption>The Output of Bitmap Implementation</figcaption></center>
</figure>

<br>

As we can see from the result, the amount of extra time we were left with is 1 millisecond for a specific frame with the multi cactus image. This is not a lot considering that it was about 7-8 milliseconds before implementing the graphics. Therefore, there was a pressing need to optimize the process of drawing bitmap.

#### 5. Optimization of the `tft_drawBitmap()` function

In order to optimize the function, we dug into the library and studied how each function is implemented. Particularly, the `tft_drawBitmap()` function is implemented in the following manner:
- Define a `byteWidth` and counter variables `i` & `j`.
- For all the elements in the given bitmap, check if the given pixel is set.
- If the given pixel is set, draw a pixel of the given color using the `tft_drawPixel()` function.
- Otherwise, do nothing.

At first glance, this doesn't look particularly unoptimized. However, digging in deeper, we found that calling the `tft_drawPixel()` function for all the pixels is pretty time consuming as the `tft_drawPixel()` function is implemented as follows:
- Configure the row address.
- Configure the column address.
- Send the color to be drawn with.

This row and column address configuration for all the pixels is quite time consuming and redundant. On the other hand, comparing it to the `tft_fillRect()` function, this function is implemented as follows:
- Set the boundary condition for the rectangle.
- Set the address window.
- For all the pixels, send the color to be drawn with.

This function is extremely optimized as we need to set the address window only once. Therefore, we combined the two functions to create a new function called `drawBitmap()` which takes in the same set of arguments as the `tft_drawBitmap()` function (and is therefore interchangeable). It was implemented as follows:
- Set the boundary condition for the rectangle.
- Set the address window.
- For all the elements in the given bitmap, check if the given pixel is set.
- If the given pixel is set, send the color directly to the TFT using the `tft_spiwrite16()` function.
- Otherwise, send the `0xFFFF` directly to the TFT using the `tft_spiwrite16()` function to draw a black background.

_Note: This function assumes that the background is black in color. It can be optimized to take a different background color._

The implemented function is shown below.

```c
#define pgm_read_byte(addr) (*(const unsigned char *)(addr))
void drawBitmap(short x, short y, const unsigned char *bitmap, short w, short h, unsigned short color){
    if((x >= _width) || (y >= _height)) return;
    if((x + w - 1) >= _width)  w = _width  - x;
    if((y + h - 1) >= _height) h = _height - y;

    tft_setAddrWindow(x, y, x + w - 1, y + h - 1);
    _dc_high();
    _cs_low();
  
    short i, j, byteWidth = (w + 7) / 8;
    for(j = 0; j < h; j++){
        for(i = 0; i < w; i++){
            if(pgm_read_byte(bitmap + j * byteWidth + i / 8) & (128 >> (i & 7))) {
                tft_spiwrite16(color);
            }
            else{
                tft_spiwrite16(0x0000);
            }
        }
    }

  _cs_high();
}
```

<br>

After implementing the bitmaps using our optimized function, we tested the code out again using the GUI. The result is shown below.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/ECE4760/FinalProject/Files/BitmapOptimized.png" style="width: 600px; height: 402px;" >
</div>
<figure>
    <center><figcaption>The Output of Optimized Bitmap Implementation</figcaption></center>
</figure>

<br>

As we can see from the result, the amount of extra time we were left with is 7 milliseconds for a specific frame with the multi cactus image. This is a huge improvement as compared to the unoptimized version of the code. This timing difference is even greated for larger bitmaps as there are more pixels to draw.